<a href="https://colab.research.google.com/github/201524495/201524495/blob/main/250812%EC%83%9D%EC%84%B1%EB%AA%A8%EB%8D%B8%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습 1: Text & Image 생성

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers diffusers
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from diffusers import StableDiffusionPipeline

### 필요한 모델을 먼저 로드합니다.

In [ ]:
from transformers import pipeline

# 텍스트 생성 모델
# generator = pipeline("text-generation", model="openai/gpt-oss-20b")
generator = pipeline("text-generation", model="Qwen/Qwen2.5-1.5B-Instruct")

# 이미지 생성 모델
model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")

## Stable Diffusion (확산모델) 실행

텍스트 → 이미지

    예: “푸른 하늘 아래 해바라기 밭”이라고 쓰면 그 장면을 그림으로 만들어줍니다.

오픈소스

    무료로 배포되어, 누구나 PC에 설치해 사용하거나 수정할 수 있습니다.

작동 원리

    처음엔 ‘노이즈’의 이미지에서 시작 → 글(프롬프트)을 보고 조금씩 노이즈를 지워 → 최종 그림 완성.

    이 과정을 diffusion(확산) 모델이라고 부릅니다.

필수 용어

    프롬프트: 원하는 그림을 설명하는 문장

    시드(seed): 같은 설정으로 재현 가능한 ‘랜덤 값’

    샘플러: 노이즈를 지우는 방식

In [ ]:
# model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
model.to("cuda")  # GPU 사용

In [ ]:
# Sampling Steps: 이미지 품질을 조정하는 파라미터인 num_inference_steps 설정.

prompt = "A beautiful sunset over the ocean with a calm sky"
result = model(prompt, num_inference_steps=20).images[0]
result

In [ ]:
prompt = "A surrealist painting of a lion made of clouds, with a dreamy atmosphere and soft pastel colors"
result = model(prompt, num_inference_steps=10).images[0]
result

In [ ]:
prompt = "A surrealist painting of a lion made of clouds, with a dreamy atmosphere and soft pastel colors"
result = model(prompt, num_inference_steps=50).images[0]
result

In [ ]:
prompt = "A surrealist painting of a lion made of clouds, with a dreamy atmosphere and soft pastel colors"
result = model(prompt, num_inference_steps=100).images[0]
result

In [ ]:
prompt = "A surrealist painting of a lion made of clouds, with a dreamy atmosphere and soft pastel colors"
result = model(prompt, num_inference_steps=200).images[0]
result

### Inferece Steps의 의미

In [ ]:
prompt = "A surrealist painting of a lion made of clouds, with a dreamy atmosphere and soft pastel colors"
for num_step in [1, 10, 50, 75, 100]:
  result = model(prompt, num_inference_steps=num_step).images[0]
  display(result)

## Prompt diversification

In [ ]:
# First prompt for generating a concept of 'dog'
cls_tmp = "a photo of dog"

first_stage_prompt = '''
To generate images using a text-to-image generation model, I need to create a prompt for a specific concept.
Your concept is {cls_tmp}. Keep the domain photorealistic, and use diverse visual scenes, visual styles, or color palettes.

Please create one prompt sentence (under 10 words) that fits this description.
Please ensure the response format is strictly 'prompt: answer' and include the phrase {cls_tmp}.
'''

# Following prompt for generating a concept of 'dog'
second_stage_prompt = '''
To generate images using a text-to-image generation model, I need to create a prompt for a specific concept.
Your concept is {cls_tmp}. Keep the domain photorealistic, and use diverse visual scenes, visual styles, or color palettes.

Here is a list of prompts that I have previously generated.
Please create a new prompt that does not overlap with these.

{first_stage_results}

Please create one prompt sentence (under 10 words) that fits this description.
Please ensure the response format is strictly 'prompt: answer' and include the phrase {cls_tmp}.
'''

 `generator` object로 텍스트를 생성할 수 있습니다.

In [ ]:
def get_response(prompt):
  # Generate text based on a prompt
  generated_text = generator(prompt, max_new_tokens=50, num_return_sequences=1)

  return generated_text[0]['generated_text'][len(prompt):].strip()

In [ ]:
get_response("Hello, I'm a language model,")

In [ ]:

get_response(first_stage_prompt.format(cls_tmp=cls_tmp))

In [ ]:
first_stage_results = []
prompt = first_stage_prompt.format(cls_tmp=cls_tmp)
for _ in range(2):
  response = get_response(prompt)
  print(response)
  generated_image = model(response.replace("prompt:", "").strip(), num_inference_steps=20).images[0]
  display(generated_image)

  first_stage_results.append(response)

In [ ]:
prompt = second_stage_prompt.format(cls_tmp=cls_tmp, first_stage_results=first_stage_results)
response = get_response(prompt)
print(response)
generated_image = model(response, num_inference_steps=20).images[0]
display(generated_image)

### 오픈 모델 사용하기 - from HuggingFace.
https://huggingface.co/models?pipeline_tag=text-generation&sort=trending

### Heuristics:  프롬프트 팁

[주제] + [상세 묘사] + [스타일/분위기] + [구도/시점] + [기술적 태그]

    [주제]: 무엇을 그릴지 (인물, 동물, 풍경, 사물 등)

    [상세 묘사]: 나이, 표정, 옷, 색상, 배경 요소

    [스타일/분위기]: realistic, anime, watercolor, cyberpunk, fantasy 등

    [구도/시점]: close-up, full body, aerial view, portrait 등

    [기술적 태그]: high detail, ultra realistic, 8k, cinematic lighting 등


"A cute cat wearing a wizard hat, holding a magic staff, fantasy forest background, watercolor style, high detail, soft lighting"

"Surreal dreamscape, floating islands, waterfalls falling into the sky, pastel colors, fantasy style, highly detailed"

In [ ]:
generated_image = model("A cute cat wearing a wizard hat, holding a magic staff, fantasy forest background, watercolor style, high detail, soft lighting"
, num_inference_steps=50).images[0]
generated_image


## CLIP (시각-언어모델)을 활용해서 원치않는 결과 필터링

In [ ]:
# CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# 이미지 전처리 함수
def preprocess_image(image):
    """Colab 변수 result를 CLIP 입력에 맞게 전처리"""
    return preprocess(image).unsqueeze(0).to(device)

In [ ]:
# 텍스트 리스트 준비
texts = ["a dog", "a cat", "a person running", "a beautiful landscape", "a lion"]
text_tokens = clip.tokenize(texts).to(device)

In [ ]:
# Colab 변수 result를 사용
display(result)
image_tensor = preprocess_image(result)

# CLIP 인코딩
with torch.no_grad():
    image_features = clip_model.encode_image(image_tensor)
    text_features = clip_model.encode_text(text_tokens)

# 유사도 계산 (코사인 유사도)
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarities = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# 결과 출력
for i, (text, score) in enumerate(zip(texts, similarities.squeeze(0).tolist())):
    print(f"{text}: {score:.4f}") # score가 1에 가까울수록 text-image가 잘 어울리는 쌍이라는 뜻입니다.

# 실습 2: 멀티모달 생성 결과를 평가하는 지표들

* 정성적 평가(Qualitative): 사람 평가(리커트 척도), 쌍 비교 선호도, 전문가 리뷰, 루브릭 기반 점수.
* 정량적 평가(Quantitative, 자동):

    * 텍스트: n-그램 중첩(BLEU/ROUGE), 임베딩 유사도(BERTScore)

    * 이미지: 분포 간 거리(FID), 다양성+품질 (IS), 텍스트-이미지 유사도(CLIPScore)

## 텍스트 평가 지표: BLEU, ROUGE, BERTScore

아래 지표들은 예측(predictions)과 정답(references)을 비교합니다.

* BLEU: precision 중심의 n-그램 중첩(기계번역에서 시작)
* ROUGE: recall 중심(텍스트 요약에서 시작)  https://aclanthology.org/W04-1013.pdf

* BERTScore: 문맥 임베딩을 활용한 의미 유사도 평가

In [ ]:
!pip -q install evaluate rouge_score bert-score clean-fid torch-fidelity ftfy regex tqdm git+https://github.com/openai/CLIP.git

In [ ]:
from pprint import pprint
import evaluate  # https://huggingface.co/docs/evaluate/en/index

# Sample predictions & references
predictions = [
    "The cat is on the mat.",
    "A young woman in a red kimono stands under cherry blossoms."
]
references = [
    ["The cat sits on the mat."],
    ["A girl wearing a red kimono is standing beneath cherry blossoms."]
]

# BLEU
bleu = evaluate.load("bleu")
bleu_res = bleu.compute(predictions=predictions, references=references)
print("BLEU")
pprint(bleu_res)

# ROUGE (expects flat reference strings, not nested)
rouge = evaluate.load("rouge")
rouge_res = rouge.compute(predictions=predictions, references=[r[0] for r in references])
print("\nROUGE")
pprint(rouge_res)

# BERTScore
from bert_score import score
P, R, F1 = score(predictions, [r[0] for r in references], lang="en", verbose=True)
print("\nBERTScore")
print("Precision:", float(P.mean()))
print("Recall   :", float(R.mean()))
print("F1       :", float(F1.mean()))


[Why BertScore?](https://colab.research.google.com/drive/1fs90QoAP_Z78Cb2B5fTfvvdT8FWPGpL6?usp=sharing
)

## 이미지 평가 지표: FID, IS, CLIPScore

진행 순서:
* 실제(real) 고양이 이미지 소규모 세트를 다운로드합니다.
* 실제 이미지를 변형(블러/노이즈 등)해 품질이 낮은 “합성(synthetic)” 세트를 만듭니다.
* 두 세트 간 **FID**와 **IS**를 계산합니다.
* 한 장의 이미지와 텍스트 프롬프트 간 **CLIPScore**를 계산합니다.

In [ ]:
import os, shutil, urllib.request, pathlib, random
from PIL import Image, ImageFilter
from io import BytesIO

base = pathlib.Path("multimodal_eval_data")
real_dir = base / "real"
fake_dir = base / "fake"
for d in [real_dir, fake_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Small sample of CC-friendly cat photos (Unsplash CDN)
urls = [
    "https://images.unsplash.com/photo-1518791841217-8f162f1e1131",
    "https://images.unsplash.com/photo-1511044568932-338cba0ad803",
    "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba",
    "https://images.unsplash.com/photo-1495360010541-f48722b34f7d",
    "https://images.unsplash.com/photo-1555685812-4b943f1cb0eb",
]

def dl(url, out_path, size=(512, 512)):
    # Add parameters for reasonable size and resize to a fixed size
    full = url + "?auto=format&fit=crop&w=512&q=80"
    with urllib.request.urlopen(full) as r:
        img = Image.open(BytesIO(r.read())).convert("RGB")
        # Resize without preserving aspect ratio to ensure uniform size
        img = img.resize(size)
        img.save(out_path)

# Download real images
for i, u in enumerate(urls):
    dl(u, real_dir / f"real_{i:02d}.jpg")

# Create "fake" images by degrading (blur + down/up sampling + noise-like effect)
for p in real_dir.glob("*.jpg"):
    img = Image.open(p).convert("RGB")
    small = img.resize((128, 128))
    blurred = small.filter(ImageFilter.GaussianBlur(2))
    # Resize without preserving aspect ratio to ensure uniform size
    degraded = blurred.resize((512, 512))
    degraded.save(fake_dir / p.name)

len(list(real_dir.glob("*.jpg"))), len(list(fake_dir.glob("*.jpg")))

In [ ]:
import torch_fidelity

metrics = torch_fidelity.calculate_metrics(
    input1=str(real_dir),
    input2=str(fake_dir),
    cuda=torch.cuda.is_available(),
    isc=True,              # Inception Score
    fid=True,              # Frechet Inception Distance
    kid=False,
    verbose=True,
    isc_splits=5
)
metrics


In [ ]:

import torch, clip
from PIL import Image
import requests

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Use one of the real images
sample_img_path = sorted(real_dir.glob("*.jpg"))[0]
image = preprocess(Image.open(sample_img_path)).unsqueeze(0).to(device)
display(Image.open(sample_img_path))

# Prompts to compare
texts = [
    "A cute cat lying on a bed",
    "A dog running in the park",
    "A bowl of pasta on a table"
]
text_tokens = clip.tokenize(texts).to(device)

with torch.no_grad():
    img_feat = model.encode_image(image)
    txt_feat = model.encode_text(text_tokens)
    img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)
    txt_feat = txt_feat / txt_feat.norm(dim=-1, keepdim=True)
    sims = (img_feat @ txt_feat.T).squeeze(0).tolist()

for t, s in zip(texts, sims):
    print(f"CLIP similarity: {s:.4f}  |  '{t}'")


결과 해석
- FID: 낮을수록 실제 이미지 분포와 더 가깝습니다.
- IS: 높을수록 이미지가 품질이 좋고 동시에 다양성이 높다는 의미입니다.

- CLIPScore(코사인 유사도): 높을수록 텍스트-이미지 의미 유사도(관련성)이 더 높습니다.

## 실습 문제
* 샘플 이미지를 여러분이 생성한 이미지(예: Stable Diffusion 결과물)로 교체하고 FID/IS를 다시 계산해 보세요.
* 텍스트 프롬프트를 변경한 뒤 CLIPScore를 다시 계산해 보세요 — 어떤 프롬프트가 각 이미지와 가장 잘 맞는지 확인합니다.
* 블러/노이즈는 CLIPScore, FID, IS 중 어떤 지표에 가장 큰 영향을 줄까요?

블러/노이즈는 CLIPScore, FID, IS 중 어떤 지표에 가장 큰 영향을 줄까요?

-> FID에 가장 큰 영향을 주는 경우가 많고, 그 다음이 IS, CLIPScore는 상대적으로 덜 민감한 편입니다.


FID (Fréchet Inception Distance)

    이미지의 전체 분포를 비교하므로, 블러·노이즈처럼 이미지 품질을 크게 해치는 변형에 민감합니다.

    특히, 픽셀 레벨의 구조와 세부 질감이 손상되면 Inception feature 분포가 많이 달라져서 값이 급격히 나빠집니다.

IS (Inception Score)

    이미지의 다양성과 품질을 모두 반영합니다.

    블러/노이즈로 인해 Inception 모델이 클래스 예측을 덜 확신하게 되면 점수가 떨어집니다.

    하지만 다양성이 유지되면 하락 폭은 FID보다 작을 수 있습니다.

CLIPScore

    텍스트-이미지 의미 일치를 보는 지표입니다.

    블러나 노이즈가 있어도, 큰 윤곽과 주요 패턴이 유지되면 CLIP은 여전히 “고양이”나 “강아지”처럼 의미를 잘 매칭합니다.

    그래서 품질 저하보다는 주제 변경이나 오브젝트 손실에 더 크게 반응합니다.

## References
* Hugging Face evaluate (BLEU/ROUGE)
* BERTScore (Zhang et al. 2020)
* FID (Heusel et al. 2017), IS (Salimans * et al. 2016)
* CLIP (Radford et al. 2021)